In [1]:
#Dependencies
import numpy as np     
import pandas as pd
from splinter import Browser  
from selenium import webdriver
from bs4 import BeautifulSoup as bs
import requests as req


In [2]:
# define splinter browser
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless = False)

#urls
base_url = "https://mars.nasa.gov/"                                          
nasa_url = "https://mars.nasa.gov/news/" 
browser.visit(nasa_url)

news_data = {}                                                               
mars_paragraph_text = [] 

# base URL for finding paragraph text/scrape
mars_html = browser.html
mars_soup = bs(mars_html, "html.parser")

#grab needed info par_1
news_title = mars_soup.find('div', class_="bottom_gradient").text
news_p1 = mars_soup.find('div', class_="rollover_description_inner").text

#grab needed info par_2
mars_soup_div = mars_soup.find(class_="content_title")
mars_soup_p2 = mars_soup_div.find_all('a', href=True)                                   
mars_soup_p2_url = mars_soup_p2[0]['href']                                               
mars_paragraph_url = base_url + mars_soup_p2_url 

response = req.get(mars_paragraph_url)                                          
mars_para_soup = bs(response.text, "html.parser")                                
mars_paragraphs = mars_para_soup.find(class_='wysiwyg_content')                     
All_paragraphs = mars_paragraphs.find_all('p') 

browser.quit() 


In [3]:
for paragraph in All_paragraphs:                                                 # iterates through paragraphs
    clean_paragraph = paragraph.get_text().strip()                           # extracts and cleans paragraphs    
    mars_paragraph_text.append(clean_paragraph)
    
mars_paragraph_text[0]

"On Nov. 26, NASA's InSight mission knew the spacecraft touched down within an 81-mile-long (130-kilometer-long) landing ellipse on Mars. Now, the team has pinpointed InSight's exact location using images from HiRISE, a powerful camera onboard another NASA spacecraft, Mars Reconnaissance Orbiter (MRO)."

In [4]:
#mars_paragraph_text[1]
news_data["mars_paragraph_text_1"] = mars_paragraph_text[0]

In [ ]:
#JPL Mars Space Images 
browser = Browser('chrome', **executable_path, headless = False)                                  
jpl_fullsize_url = 'https://photojournal.jpl.nasa.gov/jpeg/'                 
jpl_url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars" 

browser.visit(jpl_url)                                                       
jpl_html = browser.html                                                      
jpl_soup = bs(jpl_html, 'html.parser')

In [ ]:
feat_image_list = []                                                     

for image in jpl_soup.find_all('div',class_="img"):                          
    feat_image_list.append(image.find('img').get('src')) 
    
feat_image = feat_image_list[0]                                       
temp_list_1 = feat_image.split('-')                                       
temp_list_2 = temp_list_1[0].split('/')                                      
feat_image_url = jpl_fullsize_url + temp_list_2[-1] + '.jpg'

feat_image_url

In [ ]:
browser.quit() 

In [ ]:
#Mars Weather
browser = Browser('chrome', **executable_path, headless = False)                                
tweeter_url = 'https://twitter.com/marswxreport?lang=en'                       
browser.visit(tweeter_url)
tweeter_html = browser.html                                                    
tweeter_soup = bs(tweeter_html, 'html.parser')

weather_info_list = []                                                       

# extracts all tweets from soup
for weather_info in tweeter_soup.find_all('p',class_="TweetTextSize TweetTextSize--normal js-tweet-text tweet-text"):
    weather_info_list.append(weather_info.text.strip()) 
    
for value in reversed(weather_info_list):                                    
    if value[:3]=='Sol':                                                     
        mars_weather = value
        
mars_weather

In [ ]:
browser.quit() 

In [ ]:
#Mars Facts in df to html table
facts_url = 'https://space-facts.com/mars/' 
fact_list = pd.read_html(facts_url)

facts_df = fact_list[0]

facts_table = facts_df.to_html(header=False, index=False)                   
print(facts_table)


In [ ]:
#Mars Hemisperes
                                                                 
browser = Browser('chrome', **executable_path, headless = False)
usgs_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(usgs_url) 

usgs_html = browser.html                                                     
usgs_soup = bs(usgs_html, 'html.parser') 

hemisphere_image_urls = []                                                   

products = usgs_soup.find('div', class_='result-list')                       
hemispheres = products.find_all('div', class_='item')                        

for hemisphere in hemispheres:                                               
    title = hemisphere.find('div', class_='description')
    
    title_text = title.a.text                                                
    title_text = title_text.replace(' Enhanced', '')
    browser.click_link_by_partial_text(title_text)                           
    
    usgs_html = browser.html                                                 
    usgs_soup = bs(usgs_html, 'html.parser')                                 
    
    image = usgs_soup.find('div', class_='downloads').find('ul').find('li')  
    img_url = image.a['href']
    
    hemisphere_image_urls.append({'title': title_text, 'img_url': img_url})   
    
    browser.click_link_by_partial_text('Back') 
    
hemisphere_image_urls

In [ ]:
browser.quit()